In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import os
import glob

import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords

import re
import string

from wordcloud import WordCloud, STOPWORDS

# What exactly is the objective of this competition?

The goal is to predict whether scientific articles mention datasets. The submission must include the id of the article and a string justifying the prediction (cleaned using the ``clean_text()`` function provided). Our model must predict this string.

In [ ]:
train_0 = pd.read_json('/kaggle/input/coleridgeinitiative-show-us-the-data/train/f8b03c87-9d1a-4f20-b76b-cb6c69d447b2.json')
train_csv = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')

In [ ]:
train_csv

In [ ]:
train_csv.pub_title[0]

Here for instance, the article title "The Impact of Dual Enrollment on College Degree Attainment: Do Low-SES Students Benefit?" uses data from the National Education Longitudinal Study. Remember that we must predict the dataset(s) used for all articles or publication names. There can obviously be several datasets attached to one article.

# What exactly is our training data?

``train.csv`` is a file linking the publication and datasets referenced. We find the publication texts in the individual .json files. As for the test data, we have 4 articles to parse and link to datasets. From what I understand now, we have to find strings within the articles' text that we find are likely to be references to datasets. 

# How many different publications are there in the training set? And how many datasets are referenced?

In [ ]:
print(f'There are {len(train_csv.Id.unique())} different articles and {len(train_csv.cleaned_label.unique())} different datasets.')

# Let's plot the distribution of publications and datasets

In [ ]:
sns.countplot(x=train_csv.Id.value_counts())

In [ ]:
fig = plt.figure(figsize=(13, 6))
fig.suptitle('Distribution of articles and datasets', fontsize=20)

ax0 = plt.subplot2grid((1, 2), (0, 0))
ax1 = plt.subplot2grid((1, 2), (0, 1))

ax0.hist(train_csv.Id.value_counts())
ax0.set_xlabel("# of linked datasets by article")
ax0.set_ylabel("# of articles")

ax1.hist(train_csv.cleaned_label.value_counts())
ax1.set_xlabel("# of occurences of datasets in articles")
ax1.set_ylabel("# of datasets")
plt.show()

In [ ]:
sns.kdeplot(x=train_csv.Id.value_counts())

In [ ]:
sns.kdeplot(x=train_csv.cleaned_label.value_counts())

The distribution of linked datasets by articles is much less smoother than that of articles by datasets. **Why?** It looks like a Laplace distibution. **Could I test it?**

This script allows to check the number of files in the train directory. Taking out the train.csv file we have one 
more article than our code above found within the train.csv file.\
``cd ../../kaggle/input
cd ./coleridgeinitiative-show-us-the-data/train
ls -f . | wc -l``\
The answer should be 14318.

## What form do the articles have?

In [ ]:
article0 = pd.read_json('/kaggle/input/coleridgeinitiative-show-us-the-data/train/d0fa7568-7d8e-4db9-870f-f9c6f668c17b.json')

In [ ]:
article0

So our first article has 18 titles. The left column contains the title of the paragraph and the right column contains the text.

In [ ]:
article0.text[0]

Let's look at the paragraph titles.

In [ ]:
for sentence in article0.section_title:
    print(''.join(sentence))

# Dataset representation

In [ ]:
train_csv = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')

In [ ]:
stopwords = set(STOPWORDS)

wordcloud = WordCloud(background_color='white',
                      stopwords=stopwords,
                      max_words=100,
                      max_font_size=30,
                      scale=3,
                      random_state=1)
   
cloud = wordcloud.generate(str(train_csv['dataset_title'].unique()))

fig = plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(cloud)
plt.show()